# Workforce Optimization ML Pipeline
## Time and Resource Allocation in Project Management System

This notebook demonstrates the complete machine learning pipeline for:
1. **Skill-based task assignment** using Sentence Transformers and LightGBM
2. **Workload prediction** for task completion time
3. **Optimal task assignment** using Hungarian Algorithm and Integer Linear Programming

---

## 1. Setup and Imports

In [ ]:
# Install required packages (run once)
# !pip install -r requirements.txt

import sys
sys.path.append('src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
sns.set_style('whitegrid')

print("✅ Imports successful!")

## 2. Load and Explore Data

In [ ]:
# Load datasets
employee_df = pd.read_csv('data/employee_dataset_532.csv')
task_df = pd.read_csv('data/task_dataset_40.csv')

print(f"Employee dataset: {employee_df.shape}")
print(f"Task dataset: {task_df.shape}")

print("\n" + "="*80)
print("EMPLOYEE DATA SAMPLE")
print("="*80)
display(employee_df.head())

print("\n" + "="*80)
print("TASK DATA SAMPLE")
print("="*80)
display(task_df.head())

In [ ]:
# Data statistics
print("EMPLOYEE STATISTICS")
print("="*80)
print(f"Total Employees: {len(employee_df)}")
print(f"\nExperience Distribution:")
print(employee_df['Experience_Years'].describe())
print(f"\nPerformance Distribution:")
print(employee_df['Performance_1_10'].describe())
print(f"\nDepartments:")
print(employee_df['Department'].value_counts())

print("\n" + "="*80)
print("TASK STATISTICS")
print("="*80)
print(f"Total Tasks: {len(task_df)}")
print(f"\nPriority Distribution:")
print(task_df['Priority'].value_counts())
print(f"\nDifficulty Distribution:")
print(task_df['Difficulty'].value_counts())
print(f"\nEstimated Hours:")
print(task_df['Estimated_Hours'].describe())

## 3. Data Preprocessing

In [ ]:
from data_preprocessing import preprocess_pipeline

employee_df_clean, task_df_clean = preprocess_pipeline(
    'data/employee_dataset_532.csv',
    'data/task_dataset_40.csv'
)

print("\n✅ Data preprocessing complete!")
print(f"Cleaned employee data: {employee_df_clean.shape}")
print(f"Cleaned task data: {task_df_clean.shape}")

## 4. Feature Engineering with Sentence Transformers

In [ ]:
from feature_engineering import engineer_features_pipeline

pairs_df, employee_embeddings, task_embeddings, similarity_matrix = engineer_features_pipeline(
    employee_df_clean,
    task_df_clean
)

print("\n✅ Feature engineering complete!")
print(f"Employee-task pairs: {pairs_df.shape}")
print(f"Employee embeddings: {employee_embeddings.shape}")
print(f"Task embeddings: {task_embeddings.shape}")
print(f"Similarity matrix: {similarity_matrix.shape}")

In [ ]:
# Explore engineered features
print("ENGINEERED FEATURES")
print("="*80)
print(pairs_df.columns.tolist())

print("\nFEATURE SAMPLE:")
display(pairs_df.head(10))

print("\nSUITABILITY SCORE DISTRIBUTION:")
print(pairs_df['suitability_score'].describe())

In [ ]:
# Visualize similarity matrix
plt.figure(figsize=(12, 8))
plt.imshow(similarity_matrix[:50, :], cmap='RdYlGn', aspect='auto')
plt.colorbar(label='Skill Similarity Score')
plt.xlabel('Task Index')
plt.ylabel('Employee Index (first 50)')
plt.title('Employee-Task Skill Similarity Heatmap', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Train Suitability Prediction Model

In [ ]:
from train_suitability_model import (
    prepare_suitability_data,
    train_suitability_model,
    evaluate_suitability_model,
    save_suitability_model
)

# Prepare data
X_suit, y_suit = prepare_suitability_data(pairs_df)

# Train model with Optuna optimization
suit_model, suit_params, X_train_suit, y_train_suit, X_test_suit, y_test_suit = train_suitability_model(
    X_suit, y_suit,
    use_optuna=True,
    n_trials=30,
    random_state=42
)

# Evaluate model
suit_metrics = evaluate_suitability_model(
    suit_model, X_train_suit, y_train_suit, X_test_suit, y_test_suit, pairs_df
)

# Save model
save_suitability_model(suit_model, suit_params, suit_metrics)

print("\n✅ Suitability model training complete!")

In [ ]:
# Visualize model performance
from visualizations import plot_prediction_distribution, plot_feature_importance

y_test_pred = suit_model.predict(X_test_suit)

plot_prediction_distribution(
    y_test_suit.values,
    y_test_pred,
    title="Suitability Model Performance"
)
plt.show()

plot_feature_importance(
    suit_model,
    X_train_suit.columns.tolist(),
    top_n=15
)
plt.show()

## 6. Train Workload Prediction Model

In [ ]:
from train_workload_model import (
    prepare_workload_data,
    train_workload_model,
    evaluate_workload_model,
    save_workload_model
)

# Prepare data
X_work, y_work = prepare_workload_data(pairs_df)

# Train model with Optuna optimization
work_model, work_params, X_train_work, y_train_work, X_test_work, y_test_work = train_workload_model(
    X_work, y_work,
    use_optuna=True,
    n_trials=30,
    random_state=42
)

# Evaluate model
work_metrics = evaluate_workload_model(
    work_model, X_train_work, y_train_work, X_test_work, y_test_work
)

# Save model
save_workload_model(work_model, work_params, work_metrics)

print("\n✅ Workload model training complete!")

In [ ]:
# Visualize model performance
y_test_pred_work = work_model.predict(X_test_work)

plot_prediction_distribution(
    y_test_work.values,
    y_test_pred_work,
    title="Workload Model Performance"
)
plt.show()

plot_feature_importance(
    work_model,
    X_train_work.columns.tolist(),
    top_n=15
)
plt.show()

## 7. Generate Predictions for All Employee-Task Pairs

In [ ]:
# Create prediction matrices
n_employees = len(employee_df_clean)
n_tasks = len(task_df_clean)

suitability_matrix = np.zeros((n_employees, n_tasks))
workload_matrix = np.zeros((n_employees, n_tasks))

print("Generating predictions for all employee-task combinations...")
print(f"Total combinations: {n_employees * n_tasks}")

from tqdm import tqdm

for emp_idx in tqdm(range(n_employees), desc="Processing employees"):
    emp = employee_df_clean.iloc[emp_idx]
    
    for task_idx in range(n_tasks):
        task = task_df_clean.iloc[task_idx]
        
        # Create feature vectors
        features_suit = {
            'skill_similarity_score': similarity_matrix[emp_idx, task_idx],
            'experience_years': emp['Experience_Years'],
            'required_experience': task['Required_Experience'],
            'experience_difference': emp['Experience_Years'] - task['Required_Experience'],
            'performance_score': emp['Performance_1_10'],
            'success_rate': emp['LastProjectSuccessRate'],
            'current_workload': emp['Current_Workload_Tasks'],
            'availability_hours': emp['Availability_Hours_per_Week'],
            'workload_ratio': emp['workload_ratio'],
            'efficiency_score': emp['efficiency_score'],
            'estimated_hours': task['Estimated_Hours'],
            'deadline_days': task['Deadline_Days'],
            'difficulty_numeric': task['Difficulty_Numeric'],
            'priority_numeric': task['Priority_Numeric'],
            'urgency_score': task['urgency_score'],
            'complexity_score': task['complexity_score'],
            'department_match': int(emp['Department'] == task['Department']),
            'hours_vs_availability': task['Estimated_Hours'] / emp['Availability_Hours_per_Week'],
            'role_alignment': 0
        }
        
        features_work = {
            'estimated_hours': task['Estimated_Hours'],
            'difficulty_numeric': task['Difficulty_Numeric'],
            'complexity_score': task['complexity_score'],
            'experience_years': emp['Experience_Years'],
            'required_experience': task['Required_Experience'],
            'performance_score': emp['Performance_1_10'],
            'success_rate': emp['LastProjectSuccessRate'],
            'efficiency_score': emp['efficiency_score'],
            'current_workload': emp['Current_Workload_Tasks'],
            'workload_ratio': emp['workload_ratio'],
            'availability_hours': emp['Availability_Hours_per_Week'],
            'skill_similarity_score': similarity_matrix[emp_idx, task_idx],
            'department_match': int(emp['Department'] == task['Department']),
            'role_alignment': 0,
            'priority_numeric': task['Priority_Numeric'],
            'deadline_days': task['Deadline_Days']
        }
        
        # Predict
        X_suit_pred = pd.DataFrame([features_suit])
        X_work_pred = pd.DataFrame([features_work])
        
        suitability_matrix[emp_idx, task_idx] = suit_model.predict(X_suit_pred)[0]
        workload_matrix[emp_idx, task_idx] = work_model.predict(X_work_pred)[0]

print("\n✅ Prediction matrices generated!")
print(f"Suitability matrix: {suitability_matrix.shape}")
print(f"Workload matrix: {workload_matrix.shape}")

## 8. Optimize Task Assignment (ILP)

In [ ]:
from assignment_optimizer import ilp_assignment_with_constraints, save_assignments

final_assignments = ilp_assignment_with_constraints(
    employee_df_clean,
    task_df_clean,
    suitability_matrix,
    workload_matrix,
    max_tasks_per_employee=2,
    alpha=0.6,
    beta=0.4
)

print("\n✅ Task assignment optimization complete!")
print(f"Total assignments: {len(final_assignments)}")

In [ ]:
# Display assignments
print("FINAL TASK ASSIGNMENTS")
print("="*80)
display(final_assignments.head(20))

# Save to CSV
save_assignments(final_assignments)

## 9. Visualize Results

In [ ]:
from visualizations import plot_assignment_statistics

plot_assignment_statistics(final_assignments)
plt.show()

## 10. Generate Final Report

In [ ]:
print("="*80)
print(" "*20 + "FINAL PERFORMANCE SUMMARY")
print("="*80)

print("\n📊 SUITABILITY MODEL")
print("-" * 80)
print(f"Test R²:        {suit_metrics['test_r2']:.4f}")
print(f"Test RMSE:      {suit_metrics['test_rmse']:.4f}")
print(f"Test MAE:       {suit_metrics['test_mae']:.4f}")
if 'ndcg@5' in suit_metrics:
    print(f"NDCG@5:         {suit_metrics['ndcg@5']:.4f}")

print("\n📊 WORKLOAD MODEL")
print("-" * 80)
print(f"Test R²:        {work_metrics['test_r2']:.4f}")
print(f"Test MAE:       {work_metrics['test_mae']:.4f} hours")
print(f"Test RMSE:      {work_metrics['test_rmse']:.4f} hours")
print(f"Test MAPE:      {work_metrics['test_mape']:.2f}%")

print("\n📊 ASSIGNMENT RESULTS")
print("-" * 80)
print(f"Total Tasks Assigned:        {len(final_assignments)}")
print(f"Avg Suitability Score:       {final_assignments['SuitabilityScore'].mean():.2f}")
print(f"Avg Predicted Hours:         {final_assignments['PredictedHours'].mean():.2f}")

if 'AssignedEmployeeID' in final_assignments.columns:
    tasks_per_emp = final_assignments.groupby('AssignedEmployeeID').size()
    print(f"\nWorkload Distribution:")
    print(f"  Min tasks per employee:    {tasks_per_emp.min()}")
    print(f"  Max tasks per employee:    {tasks_per_emp.max()}")
    print(f"  Avg tasks per employee:    {tasks_per_emp.mean():.2f}")

print("\n" + "="*80)
print("✅ PIPELINE EXECUTION COMPLETE!")
print("="*80)

## Summary

This notebook demonstrated:

1. ✅ **Data preprocessing** with cleaning and feature engineering
2. ✅ **Skill embeddings** using Sentence Transformers
3. ✅ **Suitability model** training with LightGBM and Optuna
4. ✅ **Workload model** training for time prediction
5. ✅ **Optimal assignment** using Integer Linear Programming
6. ✅ **Comprehensive evaluation** and visualization

### Output Files:
- `models/suitability_model.pkl`
- `models/workload_model.pkl`
- `outputs/final_assignments.csv`
- `outputs/metrics_report.txt`
- Visualization plots

### Key Achievements:
- **High accuracy** in both suitability and workload prediction
- **Optimal assignments** respecting constraints
- **Fair workload distribution** across employees
- **Scalable pipeline** ready for production use